In [2]:
#I'm not sure why people makes their Fourier Positional Encoding so complicate and I'm not sure they are right. So here is my own version
#I might be wrong, but hey coding is understanding (or trying)

%%capture
!pip install einops

In [88]:
from torchvision.io import read_image
from einops import rearrange, reduce
import torch
import math

In [92]:
im = read_image('/content/224_224_image.jpg').to(torch.float)
print('image size:',im.size())

image size: torch.Size([3, 224, 224])


In [93]:
# Turn the image to black and white to reduce 3D dimensions to 2D dimension representing the spatial detail of the image 
black_and_white_image = reduce(im,'c h w -> h w', 'mean')
print('black and white image:',black_and_white_image.size())

black and white image: torch.Size([224, 224])


In [94]:
black_and_white_image

tensor([[232.0000, 230.0000, 229.6667,  ..., 119.3333, 128.0000, 145.0000],
        [226.0000, 229.0000, 231.6667,  ..., 122.0000, 129.0000, 146.0000],
        [222.6667, 228.6667, 230.6667,  ..., 123.3333, 129.3333, 147.3333],
        ...,
        [139.0000, 137.6667, 138.0000,  ..., 110.0000, 118.6667, 113.3333],
        [ 98.3333, 102.6667, 105.3333,  ..., 118.6667, 148.6667, 148.6667],
        [ 60.0000,  82.0000,  96.3333,  ..., 118.6667, 139.0000, 137.0000]])

In [59]:
# create a function to normalise the image array between -1 and 1
def normalisation(x):
  x =  x/x.sum(0).expand_as(x) 
  x[torch.isnan(x)]=0   #if an entire column is zero, division by 0 will cause NaNs
  x = 2*x - 1
  return x

In [98]:
#Normalise the image
normalised_image = normalisation(black_and_white_image)

In [99]:
normalised_image

tensor([[-0.9911, -0.9911, -0.9911,  ..., -0.9911, -0.9911, -0.9911],
        [-0.9911, -0.9911, -0.9911,  ..., -0.9911, -0.9911, -0.9911],
        [-0.9911, -0.9911, -0.9911,  ..., -0.9911, -0.9911, -0.9911],
        ...,
        [-0.9911, -0.9911, -0.9911,  ..., -0.9911, -0.9911, -0.9911],
        [-0.9910, -0.9910, -0.9910,  ..., -0.9911, -0.9911, -0.9911],
        [-0.9910, -0.9910, -0.9910,  ..., -0.9911, -0.9911, -0.9911]])

In [100]:
#create a log splace tensor between 1 and pi/2
fk_sin = torch.logspace(start=1, end=0.5*math.pi, steps=7, base=2)
fk_cos = torch.logspace(start=1, end=0.5*math.pi, steps=7, base=2)
pe = torch.zeros(224,224,7*2+1)

In [101]:
# Create a fourier positional encoding
for j in range(224):
  for i in range(224):
    x = normalised_image[j][i]
    sin = torch.sin(math.pi*x*fk_sin)
    cos = torch.cos(math.pi*x*fk_cos)
    fk = torch.cat((sin,cos))
    fk_ = fk.tolist()
    fk_.append(x.item())
    fk = torch.tensor(fk_)
    pe[j][i]=fk

In [102]:
pe.size()

torch.Size([224, 224, 15])

In [105]:
#Flatten the pe array:
pe = rearrange(pe,'h w f -> (h w) f')

In [107]:
pe

tensor([[ 0.0558, -0.3604, -0.7326,  ..., -0.7212, -0.9847, -0.9911],
        [ 0.0558, -0.3604, -0.7326,  ..., -0.7212, -0.9847, -0.9911],
        [ 0.0558, -0.3604, -0.7326,  ..., -0.7212, -0.9847, -0.9911],
        ...,
        [ 0.0561, -0.3601, -0.7323,  ..., -0.7209, -0.9846, -0.9911],
        [ 0.0560, -0.3601, -0.7324,  ..., -0.7210, -0.9847, -0.9911],
        [ 0.0560, -0.3601, -0.7324,  ..., -0.7210, -0.9847, -0.9911]])

In [108]:
pe.size()

torch.Size([50176, 15])

In [113]:
#Explaination:
# Along this dimension, there are 50176 inputs, which is equivalent to the flatten image array of 224 x 224
# Within each input, there are 15 values, these 15 values corresponds to 2K + 1. Where K is the number of bands. In this example, a value of 7 was used. The reason you times K by 2 is because
#you need to calculate the cos and sin value for each band frequency. You add 1 to the end because you need to concat the normalised input with the 14 fourier values (7*2+1 = 15)
print('Number of positional encoding values for each input:',pe.size(1))

Number of positional encoding values for each input: 15
